Name: Amrita Sinha Roy 
Student ID: 501306770

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/wce_endoscopy_dataset/

/content/drive/MyDrive/wce_endoscopy_dataset


In [ ]:
!unzip archive.zip

Archive:  archive.zip
replace test/0_normal/test_normal_ (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Efficient Net B2


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models

# Set directories for training, validation, and test sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Data preparation
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# EfficientNetB2 Model
def build_efficientnetb2(input_shape=(224, 224, 3), num_classes=4):
    base_model = EfficientNetB2(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model weights

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

efficientnet_model = build_efficientnetb2()
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training EfficientNetB2
epochs = 5
history = efficientnet_model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Evaluate on test data
test_loss, test_accuracy = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB2 Test Accuracy: {test_accuracy}")

# Save model
efficientnet_model.save("efficientnetb2_endoscopy.h5")


Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 767ms/step - accuracy: 0.2367 - loss: 1.4189 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 537ms/step - accuracy: 0.2446 - loss: 1.3870 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 545ms/step - accuracy: 0.2480 - loss: 1.3864 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 542ms/step - accuracy: 0.2391 - loss: 1.3863 - val_accuracy: 0.2500 - val_loss: 1.3864
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 82s 542ms/step - accuracy: 0.2538 - loss: 1.3867 - val_accuracy: 0.2500 - val_loss: 1.3863
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 367ms/step - accuracy: 0.2497 - loss: 1.3862


EfficientNetB2 Test Accuracy: 0.25


Added mean and std of ImageNet traning data for normalization

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models
import numpy as np

# Set directories for training, validation, and test sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Mean and standard deviation values for ImageNet normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Custom preprocessing function to normalize the image
def preprocess_image(img):
    # Normalize image (scale it to [0, 1], then use ImageNet mean and std)
    img = img / 255.0  # First normalize to [0, 1]
    img = (img - mean) / std  # Then apply ImageNet normalization
    return img

# Data preparation
img_size = (224, 224)
batch_size = 32

# Applying custom preprocessing using ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# EfficientNetB2 Model
def build_efficientnetb2(input_shape=(224, 224, 3), num_classes=4):
    base_model = EfficientNetB2(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model weights

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

efficientnet_model = build_efficientnetb2()
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training EfficientNetB2
epochs = 5
history = efficientnet_model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Evaluate on test data
test_loss, test_accuracy = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB2 Test Accuracy: {test_accuracy}")

# Save model
efficientnet_model.save("efficientnetb2_endoscopy.h5")


Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 104s 737ms/step - accuracy: 0.3237 - loss: 1.3800 - val_accuracy: 0.4425 - val_loss: 1.2637
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 115s 616ms/step - accuracy: 0.3882 - loss: 1.2875 - val_accuracy: 0.4975 - val_loss: 1.1938
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 82s 617ms/step - accuracy: 0.4303 - loss: 1.2343 - val_accuracy: 0.4565 - val_loss: 1.1411
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 82s 614ms/step - accuracy: 0.4557 - loss: 1.2083 - val_accuracy: 0.5085 - val_loss: 1.1785
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 64s 616ms/step - accuracy: 0.4552 - loss: 1.1763 - val_accuracy: 0.4585 - val_loss: 1.2181
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 386ms/step - accuracy: 0.4784 - loss: 1.1504


EfficientNetB2 Test Accuracy: 0.4662500023841858


Added earlystopping, reducelr on plateau

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os

# Set directories for training, validation, and test sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Mean and standard deviation values for ImageNet normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Custom preprocessing function to normalize the image
def preprocess_image(img):
    # Normalize image (scale it to [0, 1], then use ImageNet mean and std)
    img = img / 255.0  # First normalize to [0, 1]
    img = (img - mean) / std  # Then apply ImageNet normalization
    return img

# Data preparation
img_size = (224, 224)
batch_size = 32

# Applying custom preprocessing using ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# EfficientNetB2 Model
def build_efficientnetb2(input_shape=(224, 224, 3), num_classes=4):
    base_model = EfficientNetB2(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model weights

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

efficientnet_model = build_efficientnetb2()
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the ReduceLROnPlateau & ModelCheckpoint & EarlyStopping callbacks
checkpoint_path = 'model/Model.keras'
checkpoint_dir = os.path.dirname(checkpoint_path)

anne = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=1e-7)
checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, mode='min')

# Training EfficientNetB2 with callbacks
epochs = 5
history = efficientnet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[anne, checkpoint, early_stopping]
)

# Evaluate on test data
test_loss, test_accuracy = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB2 Test Accuracy: {test_accuracy}")

# Save model
efficientnet_model.save("efficientnetb2_endoscopy.h5")


Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step - accuracy: 0.3080 - loss: 1.3744
Epoch 1: val_loss improved from inf to 1.22696, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 103s 739ms/step - accuracy: 0.3084 - loss: 1.3741 - val_accuracy: 0.5055 - val_loss: 1.2270 - learning_rate: 0.0010
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.3924 - loss: 1.2771
Epoch 2: val_loss improved from 1.22696 to 1.16230, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 633ms/step - accuracy: 0.3925 - loss: 1.2769 - val_accuracy: 0.4740 - val_loss: 1.1623 - learning_rate: 0.0010
Epoch 3/5
 99/100 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step - accuracy: 0.4429 - loss: 1.2152
Epoch 3: val_loss improved from 1.16230 to 1.13052, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 666ms/step - accuracy: 0.4430 - loss: 

EfficientNetB2 Test Accuracy: 0.5412499904632568


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os

# Set directories for training, validation, and test sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Mean and standard deviation values for ImageNet normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Custom preprocessing function to normalize the image
def preprocess_image(img):
    # Normalize image (scale it to [0, 1], then use ImageNet mean and std)
    img = img / 255.0  # First normalize to [0, 1]
    img = (img - mean) / std  # Then apply ImageNet normalization
    return img

# Data preparation
img_size = (224, 224)
batch_size = 32

# Applying custom preprocessing using ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# EfficientNetB2 Model
def build_efficientnetb2(input_shape=(224, 224, 3), num_classes=4):
    base_model = EfficientNetB2(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model weights

    model = models.Sequential([
        base_model,
        # layers.GlobalAveragePooling2D(),
        # layers.Dense(128, activation='relu'),
        # layers.Dropout(0.5),
        # layers.Dense(num_classes, activation='softmax')
        layers.GaussianNoise(0.35),
        layers.GlobalAveragePooling2D(),
        layers.Dense(256,activation='relu'),
        layers.BatchNormalization(),
        layers.GaussianNoise(0.35),
        layers.Dropout(0.2),
        layers.Dense(4, activation='softmax')
    ])

    return model

efficientnet_model = build_efficientnetb2()
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the ReduceLROnPlateau & ModelCheckpoint & EarlyStopping callbacks
checkpoint_path = 'model/Model.keras'
checkpoint_dir = os.path.dirname(checkpoint_path)

anne = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=1e-7)
checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, mode='min')

# Training EfficientNetB2 with callbacks
epochs = 5
history = efficientnet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[anne, checkpoint, early_stopping]
)

# Evaluate on test data
test_loss, test_accuracy = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB2 Test Accuracy: {test_accuracy}")

# Save model
efficientnet_model.save("efficientnetb2_endoscopy.h5")


Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step - accuracy: 0.3846 - loss: 1.4974
Epoch 1: val_loss improved from inf to 1.27634, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 125s 849ms/step - accuracy: 0.3849 - loss: 1.4964 - val_accuracy: 0.4370 - val_loss: 1.2763 - learning_rate: 0.0010
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.4929 - loss: 1.1976
Epoch 2: val_loss improved from 1.27634 to 1.17216, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 100s 665ms/step - accuracy: 0.4929 - loss: 1.1974 - val_accuracy: 0.3890 - val_loss: 1.1722 - learning_rate: 0.0010
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - accuracy: 0.5132 - loss: 1.1434
Epoch 3: val_loss did not improve from 1.17216
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 646ms/step - accuracy: 0.5132 - loss: 1.1433 - val_accuracy: 0.3175 - val_loss: 1.2371 - learning_rate: 0.0010
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.5389 - loss: 1.1005
Epoch 4: val_

EfficientNetB2 Test Accuracy: 0.48875001072883606


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os

# Set directories for training, validation, and test sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Mean and standard deviation values for ImageNet normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Custom preprocessing function to normalize the image
def preprocess_image(img):
    # Normalize image (scale it to [0, 1], then use ImageNet mean and std)
    img = img / 255.0  # First normalize to [0, 1]
    img = (img - mean) / std  # Then apply ImageNet normalization
    return img

# Data preparation
img_size = (224, 224)
batch_size = 32

# Applying custom preprocessing using ImageDataGenerator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# EfficientNetB2 Model
def build_efficientnetb2(input_shape=(224, 224, 3), num_classes=4):
    base_model = EfficientNetB2(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False  # Freeze base model weights

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

efficientnet_model = build_efficientnetb2()
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the ReduceLROnPlateau & ModelCheckpoint & EarlyStopping callbacks
checkpoint_path = 'model/Model.keras'
checkpoint_dir = os.path.dirname(checkpoint_path)

anne = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=1e-7)
checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, mode='min')

# Training EfficientNetB2 with callbacks
epochs = 20
history = efficientnet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[anne, checkpoint, early_stopping]
)

# Evaluate on test data
test_loss, test_accuracy = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB2 Test Accuracy: {test_accuracy}")

# Save model
efficientnet_model.save("efficientnetb2_endoscopy.h5")


Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.2854 - loss: 1.3929
Epoch 1: val_loss improved from inf to 1.26763, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 761ms/step - accuracy: 0.2858 - loss: 1.3925 - val_accuracy: 0.3985 - val_loss: 1.2676 - learning_rate: 0.0010
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.3777 - loss: 1.2833
Epoch 2: val_loss improved from 1.26763 to 1.21894, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 638ms/step - accuracy: 0.3780 - loss: 1.2831 - val_accuracy: 0.4410 - val_loss: 1.2189 - learning_rate: 0.0010
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.4324 - loss: 1.2320
Epoch 3: val_loss improved from 1.21894 to 1.18919, saving model to model/Model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 83s 645ms/step - accuracy: 0.4325 - los

EfficientNetB2 Test Accuracy: 0.5637500286102295
